# Exemplo 10: Manufatura - Manutenção Preditiva (Dask)

Este notebook demonstra a análise de logs de vibração de máquinas industriais para prever falhas usando **Dask**.

**Cenário**: Identificar máquinas que estão operando fora dos parâmetros normais de vibração e temperatura.

## 1. Configuração

In [ ]:
# Instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar e Instalar Spark
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz && tar xf spark-3.5.0-bin-hadoop3.tgz

# Baixar e Instalar Kafka
!wget https://archive.apache.org/dist/kafka/3.6.1/kafka_2.13-3.6.1.tgz && tar xf kafka_2.13-3.6.1.tgz

# Instalar pacotes Python
!pip install -q findspark pyspark kafka-python

In [ ]:
!pip install -q dask[complete] distributed

In [ ]:
from dask.distributed import Client
client = Client()
client

## 2. Gerar Logs de Máquinas

In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

os.makedirs('manufacturing_data', exist_ok=True)

def generate_machine_logs(file_id):
    num_records = 5000
    machines = [f'MAC-{i}' for i in range(101, 151)]
    
    df = pd.DataFrame({
        'machine_id': np.random.choice(machines, num_records),
        'timestamp': pd.date_range(start='2024-01-01', periods=num_records, freq='s'),
        'vibration': np.random.normal(50, 5, num_records),
        'temperature': np.random.normal(70, 10, num_records)
    })
    
    # Simular falha na MAC-120: alta vibração e temperatura
    mask = df['machine_id'] == 'MAC-120'
    df.loc[mask, 'vibration'] = np.random.normal(80, 10, mask.sum())
    df.loc[mask, 'temperature'] = np.random.normal(95, 5, mask.sum())
    
    df.to_csv(f'manufacturing_data/log_{file_id}.csv', index=False)

print("Gerando dados...")
for i in range(10):
    generate_machine_logs(i)

## 3. Análise de Falhas

In [ ]:
ddf = dd.read_csv('manufacturing_data/log_*.csv')

# Calcular estatísticas por máquina
stats = ddf.groupby('machine_id')[['vibration', 'temperature']].agg(['mean', 'max', 'std'])

stats.columns = ['_'.join(col).strip() for col in stats.columns.values]
stats = stats.reset_index()

# Definir limiares de alerta
alerts = stats[(stats['vibration_mean'] > 60) | (stats['temperature_mean'] > 85)]

print("Máquinas em Alerta:")
print(alerts.compute())

In [ ]:
client.close()